In [1]:
import numpy as np, math, re
from itertools import product

def read_file_lines(file_name):
    with open(file_name, 'r') as file:
        lines = file.readlines()
        
        return [line.strip() for line in lines]

In [2]:
test_machines = [
  "[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}",
  "[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}",
  "[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"
]

In [3]:
def to_binary_vector(indices, size):
    vec = np.zeros(size, dtype=int)
    for i in indices:
        vec[i] = 1
    return vec

def process_machines(machines):
    s = 0
    for machine in machines:
        lights = re.search(r"\[[^\]]*\]", machine).group().strip("[]")
        lights = np.array([1 if ch == "#" else 0 for ch in lights], dtype=int)
        
        buttons = re.findall(r"\([^)]*\)", machine)
        buttons = [list(map(int, p.strip("()").split(","))) for p in buttons]
        buttons = [to_binary_vector(arr, len(lights)) for arr in buttons]

        solutions = []
        vectors = buttons
        target = lights
        
        for bits in product([0,1], repeat=len(vectors)):
            result = sum(b * v for b, v in zip(bits, vectors)) % 2
            if np.array_equal(result, target):
                solutions.append(bits)

        presses = [sum(arr) for arr in solutions]
        s += min(presses)

    return s

# process_machines(test_machines)
process_machines(read_file_lines('day10_input.txt'))

514

In [4]:
import pulp

def process_machines2(machines):
    s = 0
    for machine in machines:
        joltages = re.search(r"\{[^}]*\}", machine).group()
        joltages = [int(x) for x in joltages.strip("{}").split(",")]
        
        buttons = re.findall(r"\([^)]*\)", machine)
        buttons = [list(map(int, p.strip("()").split(","))) for p in buttons]
        buttons = [to_binary_vector(arr, len(joltages)) for arr in buttons]

        vectors = buttons
        target = joltages
        
        A = np.column_stack(vectors)
        m, n = A.shape
        
        prob = pulp.LpProblem("MinPresses", pulp.LpMinimize)
        x = [pulp.LpVariable(f"x{i}", lowBound=0, cat="Integer") for i in range(n)]
        prob += pulp.lpSum(x)
        for i in range(m):
            prob += pulp.lpSum(A[i,j]*x[j] for j in range(n)) == target[i]
        
        prob.solve()
        solution = [int(var.value()) for var in x]

        presses = sum(solution)
        s += presses

    return s

# process_machines2(test_machines)
process_machines2(read_file_lines('day10_input.txt'))

21824